# Quel est le nombre de correspondances par station ?

In [ ]:
MATCH (s:Station)-[r:CORRESPONDENCE]->(t:Station)
WITH s, COLLECT(DISTINCT t) AS CorrespondingStations
RETURN s.name AS StationName, COUNT(DISTINCT CorrespondingStations) AS NumberOfDistinctCorrespondences
ORDER BY NumberOfDistinctCorrespondences DESC;

# Quel est le nombre de stations à moins de deux kilomètres de la station LADEFENSE (on pourra prendre la distance brute sans considération de relation) ?

In [ ]:
MATCH (s:Station {name: "LADEFENSE"}), (other:Station)
WHERE s <> other AND SQRT((s.lat - other.lat)^2 + (s.lon - other.lon)^2) < 2000
RETURN COUNT(other) as StationsWithin2km

# Combien de temps faut-il pour aller en metro de LADEFENSE à CHATEAUDEVINCENNES ?  

 
Vous devez avoir installer les librairies APOC et GDS et dans le config file avoir :
dbms.security.procedures.unrestricted=jwt.security.*,apoc.*,gds.*

Executer les 4 querry suivantes

In [ ]:
MATCH (s1:Station)-[r:CONNECTS_TO]->(s2:Station)
WITH s1, s2, r, SQRT((s1.lat - s2.lat)^2 + (s1.lon - s2.lon)^2) as distance
SET r.distance = distance

In [ ]:
MATCH (s1:Station)-[r:CONNECTS_TO]->(s2:Station)
WITH s1, s2, r, r.distance as distance
SET r.time = distance / r.speed

In [ ]:
MATCH (s1:Station)-[r:WALK_TO]->(s2:Station)
WITH s1, s2, r, SQRT((s1.lat - s2.lat)^2 + (s1.lon - s2.lon)^2) as distance
SET r.distance = distance

In [ ]:
MATCH (s1:Station)-[r:WALK_TO]->(s2:Station)
WITH s1, s2, r, r.distance as distance
SET r.time = distance / r.speed

In [ ]:
// CALL gds.graph.drop('my_graph');

CALL gds.graph.project(
    'my_graph',
    {
        Station: {
            label: 'Station',
            properties: {
                line: 'line',
                lat: 'lat',
                lon: 'lon',
                traffic: 'traffic'
            }
        }
    },
    {
        CONNECTS_TO: {
            type: 'CONNECTS_TO',
            properties: {
                speed: 'speed',
                distance: 'distance',
                time: 'time'
            }
        },
        CORRESPONDENCE: {
            type: 'CORRESPONDENCE',
            properties: {
                time: 'time'
            }
        },
        WALK_TO: {
            type: 'WALK_TO',
            properties: {
                speed: 'speed',
                distance: 'distance',
                time: 'time'
            }
        }
    }
)

In [ ]:
MATCH (start:Station {name: 'LADEFENSE'}), (end:Station {name: 'CHATEAUDEVINCENNES'})
CALL gds.shortestPath.dijkstra.stream('my_graph', {
    sourceNode: start,
    targetNode: end,
    relationshipWeightProperty: 'time'
})
YIELD sourceNode, targetNode, totalCost
RETURN gds.util.asNode(sourceNode).name as source, gds.util.asNode(targetNode).name as target, totalCost as TotalMinutes;


# Combien de temps faut-il pour aller à pied de LADEFENSE à CHATEAUDEVINCENNES (on pourra considérer que tout le chemin se fait à pied, sans considération de relation) ?

In [ ]:
MATCH (a:Station {name: "LADEFENSE"}), (b:Station {name: "CHATEAUDEVINCENNES"})
WITH SQRT((a.lat - b.lat)^2 + (a.lon - b.lon)^2) as distance_directe_en_metres
RETURN distance_directe_en_metres / (4 * 1000) * 60 as temps_en_minutes


# Est-il plus rapide de faire un changement à SAINTLAZARE pour aller de MONTPARNASSEBIENVENUE à GABRIELPERI ?
nous allons créer deux chemins et comparer leur durée.

Un chemin direct entre MONTPARNASSEBIENVENUE et GABRIELPERI.
Un chemin qui passe par SAINTLAZARE.

In [ ]:
MATCH (start:Station {name: 'MONTPARNASSEBIENVENUE'}), (end:Station {name: 'GABRIELPERI'})
CALL gds.shortestPath.dijkstra.stream('my_graph', {
    sourceNode: start,
    targetNode: end,
    relationshipTypes: ['CONNECTS_TO'],
    relationshipWeightProperty: 'time'
})
YIELD totalCost
RETURN totalCost AS TotalTime

LE chemin direct on obtiens : 14.638 min

In [ ]:
// Find the shortest time between MONTPARNASSEBIENVENUE and STLAZARE
MATCH (start:Station {name: 'MONTPARNASSEBIENVENUE'}), (mid:Station {name: 'STLAZARE'})
CALL gds.shortestPath.dijkstra.stream('my_graph', {
    sourceNode: start,
    targetNode: mid,
    relationshipTypes: ['CONNECTS_TO'],
    relationshipWeightProperty: 'time'
})
YIELD totalCost
WITH MIN(totalCost) AS timeToMid

// Find the correspondence time at STLAZARE
MATCH (mid:Station {name: 'STLAZARE'})-[r:CORRESPONDENCE]->()
WITH timeToMid, AVG(r.time) AS avgCorrespondenceTime
WITH timeToMid, avgCorrespondenceTime

// Find the shortest time between STLAZARE and GABRIELPERI
MATCH (mid:Station {name: 'STLAZARE'}), (end:Station {name: 'GABRIELPERI'})
CALL gds.shortestPath.dijkstra.stream('my_graph', {
    sourceNode: mid,
    targetNode: end,
    relationshipTypes: ['CONNECTS_TO'],
    relationshipWeightProperty: 'time'
})
YIELD totalCost AS timeToEnd
WITH timeToMid, avgCorrespondenceTime, timeToEnd

// Calculate the total time
RETURN timeToMid AS Time_MONTPARNASSEBIENVENUE_to_STLAZARE, avgCorrespondenceTime AS CorrespondenceTime_at_STLAZARE, timeToEnd AS Time_STLAZARE_to_GABRIELPERI, (timeToMid + avgCorrespondenceTime + timeToEnd) AS TotalTime


LE chemin avec une correspondance = 18.433 minutes
Conclusion  :
Le chemin le plus court est le chemin direct entre MONTPARNASSEBIENVENUE et GABRIELPERI.

# Combien de stations se trouvent dans un rayon de 10 stations par train autour de STLAZARE ?
2 querry : une avec les noms des stations et la seconde avec le nombre

In [ ]:
MATCH (start:Station {name: 'STLAZARE'})-[:CONNECTS_TO*1..10]->(other:Station)
WHERE start <> other
WITH DISTINCT other.name AS DistinctStationNames
RETURN DistinctStationNames AS StationsName

In [ ]:
MATCH (start:Station {name: 'STLAZARE'})-[:CONNECTS_TO*..10]->(other:Station)
WHERE start <> other
WITH collect(DISTINCT other.name) AS DistinctStations
RETURN SIZE(DistinctStations) AS NumberOfDistinctStations

Avec shortestPath :

In [ ]:
MATCH (start:Station {name: 'STLAZARE'})
MATCH path = shortestPath((start)-[:CONNECTS_TO*..10]-(other:Station))
WHERE LENGTH(path) <= 10 AND start <> other
RETURN DISTINCT other.name as StationsName

In [ ]:
MATCH (start:Station {name: 'STLAZARE'})
MATCH path = shortestPath((start)-[:CONNECTS_TO*..10]-(other:Station))
WHERE LENGTH(path) <= 10 AND start <> other
RETURN COUNT(DISTINCT other.name) as Nombre_Stations

# Combien de stations se trouvent dans un rayon de 20 minutes par train autour de STLAZARE ?

In [ ]:
MATCH (start:Station {name: 'STLAZARE'})
MATCH (end:Station) WHERE start <> end
CALL gds.shortestPath.dijkstra.stream('my_graph', {
    sourceNode: start,
    targetNode: end,
    relationshipWeightProperty: 'time'
})
YIELD sourceNode, targetNode, totalCost
WHERE totalCost <= 20
RETURN COUNT(DISTINCT targetNode) AS NumberOfStations
